### updateECRDatastorePersonID
This is the 1st step to update the ECR datastore after receiving new MPI data from LAC.

This notebook joins the ECR datastore (`ecr`) and Patient table from the Master Patient Index (`patient`) on `patient_id` to update the ECR datastore's `person_id` with the most up-to-date `person_id` in the `patient` table. 

In [ ]:
pip install psycopg2-binary azure-keyvault

In [ ]:
from azure.identity import ManagedIdentityCredential
from azure.core.credentials import AccessToken
from azure.keyvault.secrets import SecretClient
import psycopg2
import time

from delta.tables import *
from pyspark.sql.functions import *

In [ ]:
ECR_DELTA_TABLE_FILE_PATH = "ecr-datastore"


# # Database connection parameters
# DB_NAME = "DibbsMpiDB"
# DB_USER = "postgres"
# DB_HOST = "phdidevmpi9d194c64.postgres.database.azure.com"
# DB_PORT = "5432"
# DB_TABLE = "patient"

# Set up authentication
class spoof_token:
    def get_token(*args, **kwargs):
        return AccessToken(
            token=mssparkutils.credentials.getToken(audience="vault"),
            expires_on=int(time.time())+60*10 # some random time in future... synapse doesn't document how to get the actual time
        )

credential = ManagedIdentityCredential()
credential._credential = spoof_token() # monkey-patch the contents of the private `_credential`

# Set your Key Vault information
KEY_VAULT_URL = "https://$KEY_VAULT.vault.azure.net"
DB_PASS_SECRET_NAME = "mpi-db-password"

# Create a SecretClient to interact with the Key Vault
secret_client = SecretClient(vault_url=KEY_VAULT_URL, credential=credential)

# Retrieve the secret
db_pass_secret = secret_client.get_secret(DB_PASS_SECRET_NAME)

# Database connection parameters
DB_NAME = "DibbsMpiDB"
DB_USER = "postgres"
DB_HOST = "$MPI_DB_HOST"
DB_PORT = "5432"
DB_TABLE = "patient"

# Get the secret value (password) from the previous step
db_password = db_pass_secret.value

# Connect to the database
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=db_password,
    host=DB_HOST,
    port=DB_PORT
)

# Create a cursor
cur = conn.cursor()

# Execute the query to get the list of tables in the database
cur.execute(f"""
    SELECT patient_id,person_id
    FROM {DB_TABLE};
""")

# Fetch the results
data = cur.fetchall()

# Close the cursor and connection
cur.close()
conn.close()


# Prep the MPI data for merging with ECR data 
columns=['patient_id','person_id']
patient = spark.createDataFrame(data = data, schema = columns)


# Load ecr Delta table
ecr = DeltaTable.forPath(spark,ECR_DELTA_TABLE_FILE_PATH)

# Update ecr data with `person_id` from MPI by joining on `patient_id`
ecr.alias("ecr") \
  .merge(
    patient.alias("mpi_patient"),
    "ecr.patient_id = mpi_patient.patient_id") \
  .whenMatchedUpdate(set = { "person_id": "mpi_patient.person_id", "person_id_date_added": date_format(current_timestamp(), 'yyyy-MM-dd')}) \
  .execute()

